In [73]:
import requests
import pandas as pd
import concurrent
from time import time
import sqlite3
from sqlalchemy import create_engine
import json
import ast
import pandasql as ps
from fuzzywuzzy import fuzz

In [74]:
url="https://random-data-api.com/api/users/random_user?size=100" #sine API support max 100 records in each request
userdata=[]
userfile=r"C:\Users\akash.masney\Downloads\technicalexamsr_etldevakashshrikantmasney\userdata.csv"  

# To get data in parallel from API 

In [75]:
def GetUserData(url):
    return requests.get(url).json()

starttime=time()

with concurrent.futures.ThreadPoolExecutor() as executor: # optimally defined number of threads
    res = [executor.submit(GetUserData, url) for _ in range(10)]
    concurrent.futures.wait(res) 
    
userdata=[]
for i in range(len(res)):
    userdata.extend(res[i].result())
    
df_user=pd.DataFrame(userdata)
df_user.to_csv(userfile,index=False)

print("timer taken:",time()-starttime)

timer taken: 4.1646552085876465


In [76]:
userdf=pd.read_csv(userfile)

In [77]:
chunk_size=500
batch_no=1

csv_database = create_engine(r'sqlite:///C:\Users\akash.masney\Downloads\technicalexamsr_etldevakashshrikantmasney\csv_database.db')

for chunk in pd.read_csv(userfile,chunksize=chunk_size,iterator=True):
    chunk.to_sql('usertable',csv_database, if_exists='append')
    batch_no+=1
    print('index: {}'.format(batch_no))

index: 2
index: 3


In [78]:
pd.read_sql_query("select count(1) from usertable",csv_database)

,count(1)
0,3000


# flatten the records in df first

In [79]:
def flatten_column(df,col):
    try:
        df[col]=df[col].apply(lambda d : ast.literal_eval(d))
        for newcol in df[col][0].keys():
            df[newcol]=df[col].apply(lambda d : d[newcol])
        df.drop(col,inplace=True,axis=1)
        #return df
    except Exception as e:
        print("Error in function flatten_column : ",str(e))

In [80]:
flatten_column(userdf,'subscription')
flatten_column(userdf,'employment')

In [81]:
def getaddressfields(df,col):
    try:
        df[col]=df[col].apply(lambda d : ast.literal_eval(d))
        for newcol in ['city','zip_code','state','country']:
            df[newcol]=df[col].apply(lambda d : d[newcol])
        df.drop(col,inplace=True,axis=1)
        #return df
    except Exception as e:
        print("Error in function getaddressfields : ",str(e))

In [82]:
getaddressfields(userdf,'address')

In [83]:
userdf.head()

,id,uid,password,first_name,last_name,username,email,avatar,gender,phone_number,...,plan,status,payment_method,term,title,key_skill,city,zip_code,state,country
0,5580,6b4385ea-7c72-4cd8-8d41-1638b0931cea,X0lGnRUoBb,Elijah,Hermann,elijah.hermann,elijah.hermann@email.com,https://robohash.org/quiplaceatillum.png?size=...,Polygender,+41 656.053.7214 x8728,...,Student,Idle,WeChat Pay,Payment in advance,Administration Assistant,Technical savvy,North Hermilamouth,38044-7452,Kentucky,United States
1,2414,be3df771-e9a5-4b5f-8c0e-cfd19f274a39,NBP6lKuJOC,Hoyt,Casper,hoyt.casper,hoyt.casper@email.com,https://robohash.org/sintomnisut.png?size=300x...,Female,+1-671 263-678-6425 x1611,...,Professional,Idle,Credit card,Payment in advance,National Producer,Self-motivated,New Darius,72801,Colorado,United States
2,3370,17d3f918-6e28-400e-890a-4e95d34b5587,aM7dfhVort,Alonzo,Lehner,alonzo.lehner,alonzo.lehner@email.com,https://robohash.org/quodpariaturdolores.png?s...,Genderqueer,+967 (832) 364-4457 x87267,...,Business,Active,Google Pay,Annual,Marketing Supervisor,Networking skills,New Stacy,73282,Illinois,United States
3,97,e3d918f1-77c6-404a-8449-c49806013c76,70rL1mv2Qe,Lucas,Adams,lucas.adams,lucas.adams@email.com,https://robohash.org/namnemoet.png?size=300x30...,Agender,+262 314.428.7343,...,Basic,Blocked,WeChat Pay,Payment in advance,Administration Architect,Confidence,West Erinstad,70194-5224,Connecticut,United States
4,9049,cd41952f-3366-4c2a-85d1-5fe3a44eab72,UcmCpGqesu,Marlon,Purdy,marlon.purdy,marlon.purdy@email.com,https://robohash.org/etaliquamqui.png?size=300...,Female,+673 1-163-643-7065,...,Platinum,Idle,Alipay,Full subscription,Sales Administrator,Proactive,Kundeland,32976-6832,South Carolina,United States


In [84]:
mysql='''
        SELECT
          state,
          COUNT(id) AS 'user_in_state' 
        FROM
          userdf
        GROUP BY 
          state
        ORDER BY 
          user_in_state DESC limit 10
    '''

In [85]:
ps.sqldf(mysql)

,state,user_in_state
0,Connecticut,29
1,Hawaii,28
2,Rhode Island,27
3,Virginia,26
4,Ohio,26
5,Mississippi,26
6,Alabama,25
7,Arkansas,24
8,Utah,23
9,North Dakota,23


In [86]:
userdf.columns

Index(['id', 'uid', 'password', 'first_name', 'last_name', 'username', 'email',
       'avatar', 'gender', 'phone_number', 'social_insurance_number',
       'date_of_birth', 'credit_card', 'plan', 'status', 'payment_method',
       'term', 'title', 'key_skill', 'city', 'zip_code', 'state', 'country'],
      dtype='object')

In [87]:
for i in range(1,len(userdf.index)):
    similarity=fuzz.ratio(userdf['payment_method'][i-1],userdf['payment_method'][i])
    if similarity>50:
        print(i)
        break

9


In [88]:
userdf.corr()

,id,social_insurance_number
id,1.00000,-0.04918
social_insurance_number,-0.04918,1.00000


In [89]:
userdf['payment_method'].value_counts()

Debit card        99
WeChat Pay        90
Credit card       88
Apple Pay         86
Visa checkout     86
Money transfer    85
Alipay            83
Paypal            82
Cheque            80
Google Pay        80
Cash              76
Bitcoins          65
Name: payment_method, dtype: int64

In [90]:
userdf['gender'].value_counts()

Genderfluid    147
Female         135
Male           131
Genderqueer    128
Non-binary     126
Bigender       123
Agender        118
Polygender      92
Name: gender, dtype: int64

In [91]:
cols_to_drop=['id','uid','first_name','last_name','password','email','avatar','social_insurance_number','credit_card','phone_number','city','zip_code','country']
userdf_clean=userdf.drop(cols_to_drop,inplace=False,axis=1)

In [93]:
userdf_clean['date_of_birth']=pd.to_datetime(userdf_clean['date_of_birth'])

In [94]:
userdf_clean.head()

,username,gender,date_of_birth,plan,status,payment_method,term,title,key_skill,state
0,elijah.hermann,Polygender,1972-09-08,Student,Idle,WeChat Pay,Payment in advance,Administration Assistant,Technical savvy,Kentucky
1,hoyt.casper,Female,1969-02-02,Professional,Idle,Credit card,Payment in advance,National Producer,Self-motivated,Colorado
2,alonzo.lehner,Genderqueer,1996-03-13,Business,Active,Google Pay,Annual,Marketing Supervisor,Networking skills,Illinois
3,lucas.adams,Agender,1965-10-22,Basic,Blocked,WeChat Pay,Payment in advance,Administration Architect,Confidence,Connecticut
4,marlon.purdy,Female,2003-05-03,Platinum,Idle,Alipay,Full subscription,Sales Administrator,Proactive,South Carolina


In [102]:
from datetime import datetime

userdf_clean['age']=pd.Timestamp.now().normalize()-userdf_clean['date_of_birth']


In [103]:
userdf_clean['age']

0     18211 days
1     19525 days
2      9624 days
3     20724 days
4      7017 days
         ...    
995   18196 days
996   15007 days
997    8416 days
998   22369 days
999   18937 days
Name: age, Length: 1000, dtype: timedelta64[ns]